### Bob hat einen Freund Namens Charlie. Bob möchte das die Coins welche er von Alice geschickt bekommt NUR ausgegeben werden können wenn Bob und Charlie die Transaktion mit ihren jeweils eigenen privaten Schlüsseln signieren. Alice muss daher die Coins nicht an den öffentlichen Schlüssel von Bob schicken sondern an ein Skript welches die beiden öffentlichen Schlüssel von Bob und Charlie enthält. Diese Art der Transaktion haben wir bereits als **P2MS** oder **Bare Multisig** kennengelernt. P2MS hat jedoch zwei entscheidenden Nachteile.
    1. Je mehr mögliche öffentliche Schlüssel für MultiSig genutzt werden desto länger wird der ScriptPubKey der Lockingtransaktion und damit wird auch die Transaktion selbst immer größere und teurer.
    2. Die Kosten für die extrem große Lockingtransaktion muss der Sender bezahlen. Das ist aber ökonomisch unfair. Wieso soll der Sender den Preis dafür zahlen, dass der Empfänger einen aufwendigeren Mechanismus zur Ausgabe der Coins bekommt.
Die Lösung beider Probleme besteht darin die Coins an den Hash eines Skripts zu schicken. Der Hash ist immer gleich lang. Daher ändert sich für den Sender nicht die Transaktionsgröße je mehr öffentliche SChlüssel dazu kommen. Das Skript und damit auch die größere Datenmenge wird erst beim Verwenden der Coins veröffentlicht und muss daher vom Eigentümer der MultiSig-Transaktion bezahlt werden.
Diese Art der Transaktion wird als **P2SH** bezeichnet. 

In [ ]:
#bitcoind -regtest -daemon -fallbackfee=0.0000025 -> startet bitcoin in regtest mode
#bcr loadwallet alice (oder bcr createwallet alice)
#bcr loadwallet bob (oder bcr createwallet bob)
#bcr loadwallet charlie (oder bcr createwallet charlie)
#bcr -rpcwallet=alice getnweaddress "" legacy -> gibt die addresse zur Vorbereitung von welcher dann die P2SH-Locking Transaktion ausgeht
#bcr -rpcwallet=alice sendtoaddress <address> <amount in btc>
#bcr -rpcwallet=alice generatetoaddress <amount of blocks> <address>
from io import BytesIO

from src.transaction import CTx, CTxIn, CTxOut
from src.script import Script, p2pk_script, p2ms_script, p2sh_script
from src.ecdsa import PrivateKey
from src.crypto import hash160

**1.** Zunächst müssen Bob und Charlie das von ihnen gewünschte Script erzeugen.

In [ ]:
bob_private_key_wif_format = "cRmJMCZ5ShFBKaR4wKmoC7NJazFFK4ibwVVpjxa9s3ZMaJyMiTd9"
bob_private_key = PrivateKey.convert_wif_format(bob_private_key_wif_format)
bob_private_key_int_format = bob_private_key.get_private_key_int()
bob_public_key = bob_private_key.get_public_key()

charlie_private_key_wif_format = "cVJuiR5EMMNfzvdLQw5nGJFmDQ7TW1QsXx2NYQJWMF7zve17GHNr"
charlie_private_key = PrivateKey.convert_wif_format(charlie_private_key_wif_format)
charlie_private_key_int_format = charlie_private_key.get_private_key_int()
charlie_public_key = charlie_private_key.get_public_key()

bob_and_charlie_skript = p2ms_script([bob_public_key.sec_format(), charlie_public_key.sec_format()], 2, 2)
bob_and_charlie_skript_hash160 = hash160(bytes.fromhex(bob_and_charlie_skript.serialize_script().hex()[2:]))

**1.1** In der ersten Transaktion werden von Alice die 10000 Satoshis an den von Bob kontrollierten öffentlichen Schlüssel versendet. Um das notwendige Transaktionsformular ausfüllen zu können, benötigt Alice folgende Informationen. 

In [ ]:
#Transaktionsinputdaten
#bcr -rpcwallet=alice dumpprivkey <alice address> -> gibt den zu address gehörigen privaten Schlüssel in WIF
alice_private_key_wif_format = "cTWa7GGJGWrW9AUZBsyKFCUKeWyMzgKAyotZe5MdZ7QNYKmXz28u"
alice_private_key = PrivateKey.convert_wif_format(alice_private_key_wif_format)
alice_private_key_int_format = alice_private_key.get_private_key_int()
alice_public_key = alice_private_key.get_public_key()

#bcr -rpcwallet=alice listunspent -> listet alle utxo's von alice auf
#bcr -rpcwallet=alice gettransaction <tx_id> -> gibt das "Überweisungsformular" der Transaktion mit der angegebenen ID zurück.
alice_previous_tx_id_to_spent = "b320b5307f5abda19b69653f422487f8d71a72994bf48eda9972ea43808a791e"
alice_previous_tx_index_to_spent = 1
alice_previous_script_pub_key_to_spent = "76a914a6ae4ca0c59722aa47f662ace1d327af89cf3d5d88ac"

#Transaktionsoutputdaten
alice_amount_to_spent = 10000

**1.2** Nun kann Alice die Lockingtransaktion erzeugen. Um die Satoshis an Bob's öffentlichen Schlüssel zu senden, muss Alice im Output der Transaktion ein scriptPubKey der Form: OP_HASH160 <script_hash> + OP_EQUAL generieren.

In [ ]:
transaction_input = CTxIn(bytes.fromhex(alice_previous_tx_id_to_spent), alice_previous_tx_index_to_spent, script_sig=Script())

alice_script_pubkey = p2sh_script(bob_and_charlie_skript_hash160)
transaction_output = CTxOut(alice_amount_to_spent, alice_script_pubkey)

alice_locking_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Alice unsignierte P2SH Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

**1.3** Nun muss Alice die Lockingtransaktion noch signieren. D.h. sie muss nachweisen, dass sie aus der vorhergehenden Transaktion berechtigt ist die 10000 Satoshis auszugeben.

In [ ]:
alice_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(alice_previous_script_pub_key_to_spent)//2)[2:]}{alice_previous_script_pub_key_to_spent}")))
alice_locking_transaction.sign_transaction(0, [alice_private_key_int_format], alice_script_sig)

#bcr -rpcwallet=alice testmempoolaccept '["tx"]' -> prüft ob die Transaktion alle Checks für die Aufnhame in den Mempool besteht
#bcr -rpcwallet=alice decoderawtransaction <tx> -> decodiert die serilaisierte Transaktion ins "Überweisungsformular"
#bcr -rpcwallet=alice sendrawtransaction <tx> -> übermittelt schließlich die Transaktion im Netzwerk so dass sie von minern für einen block aus dem mempool aufgenommen werden kann

print(f"Alice signierte P2SH Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

**2.** Nun möchten Bob und Charlie die an den Hash ihres Scripts gebundenen Satoshis weiter versenden. Dazu müssen die beiden im Inputfeld (in dem auf den eben von Alice erzeugten Output referenziert wird), zum Beweis das sie berechtigt sind den Output auszugeben, ihre Signaturen sowie das von ihnen erzeugte Skript vorlegen. Die Reihenfolge in der die Signaturen vorliegen muss die selbe Reihenfolge sein in der die öffentlichen Schlüssel im Skript vorliegen. Damit Bob das notwendige Transaktionsformular ausfüllen kann benötigt er folgende Informationen.

In [ ]:
#bcr decoderawtransaction <raw_tx_hex>  parsed raw tx

bob_previous_tx_id_to_spent = "c8542e73ffed2d5a3811c1cde01227177b011adf5781f7124e3f7534f9a98518"
bob_previous_tx_index_to_spent = 0
bob_previous_script_pub_key_to_spent = "a9142aa944882aeab4b3860ad6ea9aa8d6c53b83009887"

#Transaktionsoutputdaten
bob_amount_to_spent = 9000
#bob's reciever address is alice public key

**2.1** Nun kann Bob die Transaktion erstellen um die eben von Alice erhaltenen Satoshis weiter zu versenden.

In [ ]:
transaction_input = CTxIn(bytes.fromhex(bob_previous_tx_id_to_spent), bob_previous_tx_index_to_spent, script_sig=Script())

script_pubkey = p2pk_script(alice_public_key.sec_format())
transaction_output = CTxOut(bob_amount_to_spent, script_pubkey)

bob_spending_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Bob unsignierte P2SH Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")

**2.2** Nun können Bob und Charlie diese Transaktion mit ihren privaten Schlüsseln signieren und versenden.

In [ ]:
bob_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(bob_previous_script_pub_key_to_spent)//2)[2:]}{bob_previous_script_pub_key_to_spent}")))

bob_spending_transaction.sign_transaction(0, [bob_private_key_int_format, charlie_private_key_int_format], bob_script_sig, 2, 2, bob_and_charlie_skript)

#bcr -rpcwallet=bob testmempoolaccept '["tx"]'
#bcr -rpcwallet=bob decodetransaction <tx>
#bcr -rpcwallet=bob sendrawtransaction <tx>

print(f"Bob signierte P2SH Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")